In [39]:
import argparse
import cv2
import os
import logging
import uuid
import math
import numpy as np
import matplotlib.pyplot as plt

from scipy.stats import skew
from utilities.AutomaticSegmentation import *

In [59]:
# n = len(os.listdir("TestImages/bank"))
# c = 2
# r = (n // 2) + 1

# GAUSSIAN???
# kernel = np.ones((16, 16))

BOUNDARY_EXPAND = 30 # px
MAX_DILATION_ROUNDS = 5
CC_CONNECTIVITY = 8  
kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE,(16, 16))

base_path = "../TestImages/tricky"
for impath in os.listdir(base_path):
    
    path_to_image = "{}/{}".format(base_path, impath)
    img = cv2.imread(path_to_image, cv2.IMREAD_GRAYSCALE)
    seed_pt, ref_pt, enh = get_ROI(img)

    enh_gray = (enh*255).astype(np.uint8)
    
#     I_rc = cv2.bitwise_not(cv2.morphologyEx(
#         enh_gray, 
#         cv2.MORPH_OPEN, 
#         kernel))

    I_rc = cv2.morphologyEx(
        enh_gray, 
        cv2.MORPH_CLOSE, 
        kernel)
   
    otsu_thresh, img_morph = cv2.threshold(I_rc, 0,255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)
    
#     plt.figure()
#     plt.title(impath)
#     plt.imshow(img_morph)
#     plt.colorbar()
#     plt.scatter(x=ref_pt[1], y=ref_pt[0], s=40, c='r')
    
    im2, contours, hierarchy = cv2.findContours(
        img_morph,
        cv2.RETR_TREE,
        cv2.CHAIN_APPROX_SIMPLE)
    
    br = [cv2.boundingRect(c) for c in contours]
    
    print("Seed point: {}".format(seed_pt))
    
    for r in br:
        x,y,w,h = r
        if (
        seed_pt[1] > x and seed_pt[1] < x + w and
            seed_pt[0] > y and seed_pt[0] < y + h
        ):
            cv2.rectangle(img,(x-25,y-25),(x+w+25,y+h),(0,255,0),2)
            
    plt.scatter(x=seed_pt[1], y=seed_pt[0], s=40, c='r')
    plt.imshow(img)
        
#     br = [(b, idx) for idx, b in enumerate(br) if (
#         (b[0] < seed_pt[1]) and 
#         (seed_pt[1] < (b[0] + b[2])) and
#         (b[1] < seed_pt[0]) and 
#          (seed_pt[0] < (b[1] + b[3]))
#     )]
    break
    
    areas = [b[0][2] * b[0][3] for b in br]
    min_rect, cont_ind = br[np.argmin(areas)]
    
    print(min_rect)
    
    x,y,w,h = min_rect
    cv2.rectangle(img,(x,y),(x+w,y+h),(0,255,0),2)
#     cv2.drawContours(img, contours, cont_ind, (0,255,0), 3)
    plt.imshow(img)
    print(len(contours))


Seed point: [284.57624758 290.89125404]


In [ ]:
def __determine_bounding_region(image, seed_point):
    
    I_rc = cv2.bitwise_not(cv2.morphologyEx(
        image, 
        cv2.MORPH_OPEN, 
        kernel))
    
    plt.imshow(I_rc)
    img_morph = cv2.threshold(I_rc, 0,255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)
    
    # You need to choose 4 way simple connectivity
    connectivity = 4  
    # Perform the operation
    output = cv2.connectedComponentsWithStats(img_morph, connectivity, cv2.CV_32S)
    # Get the results
    # The first cell is the number of labels
    num_labels = output[0]
    # The second cell is the label matrix
    labels = output[1]
    # The third cell is the stat matrix
    stats = output[2]
    # The fourth cell is the centroid matrix
    centroids = output[3]
    
    print(labels)
    print(stats)

In [ ]:
img = cv2.imread("TestImages/bank/b39c6fb0-0f53-46d8-a0f2-455870676378.png", cv2.IMREAD_GRAYSCALE)
rp, enh = get_ROI(img)
max_p, post_cands = __get_seed_point(enh, rp, nd=12, h=20, eps=1)

kernel = np.ones((16, 16),np.uint8)
# I_ro = cv2.erode(img, kernel = se, iterations = 1)
# I_ro_c = cv2.bitwise_not(I_ro)

# I_rc = cv2.bitwise_and(I_ro_c, I_ro_c, mask = cv2.bitwise_not(cv2.dilate(I_ro, kernel = se, iterations = 1)))
I_rc = cv2.bitwise_not(cv2.morphologyEx(img, cv2.MORPH_OPEN, kernel))

ret3,th3 = cv2.threshold(I_rc, 0,255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)
print(ret3)
plt.imshow(th3)
plt.colorbar()

In [4]:
%matplotlib

Using matplotlib backend: MacOSX
